In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pymongo import MongoClient
import os
import warnings
import sys
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

PLOTS_PATH = os.path.join(library_path, 'plots')

print("Libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["Diagnosis_Severity_PD_Voice"]
collection = db["studies"]

print("🔄 Loading studies from MongoDB...")
fields_to_extract = {
    "doi"             : 1, 
    "year"            : 1, 
    "study_id"        : 1,
    "ml_approaches"   : 1,
    '_id'             : 0
}  # 1 = include, 0 = exclude
studies_cursor = collection.find({}, fields_to_extract)
studies_list = list(studies_cursor)


print(f"📊 Total studies loaded: {len(studies_list)}")
print(f"📄 Sample document keys: {list(studies_list[0].keys()) if studies_list else 'No documents found'}")

In [ ]:
def get_feature_extraction(experiment_list:list)->list:

    feat_extract = []

    for experiment in experiment_list:
        extraction_pipe = experiment.get('feature_extraction')
        if extraction_pipe is None:
            extraction_pipe = []
        feat_extract += extraction_pipe

    return list(set(feat_extract))

def experiments_with_feat_ext(experiment_list: list)->int:

    count = 0
    for experiment in experiment_list:
        if experiment.get('feature_extraction') is not None:
            count+=1
    return count

In [ ]:
experiment_df = pd.DataFrame(studies_list)
experiment_df['num_experiments'] = experiment_df['ml_approaches'].apply(lambda x: len(x))
experiment_df['exp_with_feat_extract'] = experiment_df['ml_approaches'].apply(lambda x: experiments_with_feat_ext(x))
experiment_df.head()

In [ ]:
num_experiments = experiment_df['num_experiments'].sum()
experiments_with_feat_ext = experiment_df['exp_with_feat_extract'].sum()
print(f"Total number of experiments: {num_experiments}")
print(f"Number of experiments with feature extracttion: {experiments_with_feat_ext}")
print(f"NUmber of exoeriments with out feature extraction: {num_experiments-experiments_with_feat_ext}")
print(f"Percentage of experiments with feature extraction: {experiments_with_feat_ext/num_experiments*100:.2f}%")

In [ ]:
def clean_technique_name(technique):
    """
    Clean technique names by extracting the base strategy.
    Handles both colon-separated and parentheses-separated details.
    """
    if not isinstance(technique, str):
        return str(technique)
    
    # Remove leading/trailing whitespace
    cleaned = technique.strip()
    
    # Handle specific name standardizations first
    if cleaned == "Short Term Fourier Transform" or cleaned == "Short Time Fourier Transform" or cleaned == "Short-Term Fourier Transform":
        cleaned = "Short-Time Fourier Transform"

    if cleaned == "Wave2Vec2" or cleaned == "Wav2Vec" or cleaned == "Wave2Vec2.0" or cleaned == "Wav2Vec2":
        cleaned = "Wave2Vec"
    if "wav2vec" in cleaned.lower():
        cleaned = "Wave2Vec"

    if cleaned == "Articulation features":
        cleaned = "Articulation Features"

    if "resnet" in cleaned.lower():
        cleaned = "ResNet"

    if "Phonation features" == cleaned:
        cleaned = "Phonation Features"
    
    # Spectrogram harmonization
    if (cleaned == "Spectograms" or 
        cleaned == "Spectrogram-color" or 
        cleaned == "Spectrogram-grayscale" or 
        cleaned == "Gray Scale Spectrogram" or
        cleaned == "Color Spectrogram"):
        cleaned = "Spectrogram"
    
    # Mel-Spectrogram harmonization
    if cleaned == "Mel-Spectogram" or cleaned == "Mel-Spectrogram" or cleaned == "Mel Spectrogram" or cleaned == "Mel Spectrograms":
        cleaned = "Mel-Spectrogram"
    
    # MFCC harmonization
    if (cleaned == "Mel Frequency Cepstral Coefficients" or 
        cleaned == "Mel-Frequency Cepstral Coefficients-mean" or 
        cleaned == "Mel-Frequency Cepstral Coefficients-variance" or 
        cleaned == "Mel-Frequency Cepstrum Coefficients"):
        cleaned = "Mel-Frequency Cepstral Coefficients"

    if "First Three Formants" in cleaned:
        cleaned = "First Three Formants"

    # Handle colon-separated details (e.g., "Wavelet Transform: Daubechies")
    if ':' in cleaned:
        cleaned = cleaned.split(':')[0].strip()
    
    # Handle parentheses details (e.g., "MFCC (Mel-Frequency Cepstral Coefficients)")
    if '(' in cleaned:
        cleaned = cleaned.split('(')[0].strip()
    
    # Handle bracket details (e.g., "FFT [Fast Fourier Transform]")
    if '[' in cleaned:
        cleaned = cleaned.split('[')[0].strip()
    
    # Additional cleanup for common patterns
    # Remove trailing dashes or other separators
    cleaned = cleaned.rstrip(' -–—_')
    
    return cleaned

In [ ]:
experiment_df['feat_extraction'] = experiment_df['ml_approaches'].apply(
    lambda x: get_feature_extraction(x)
)

extraction_df = experiment_df.explode('feat_extraction')
extraction_df.head(7)

In [ ]:
extraction_df['cleaned_technique'] = extraction_df['feat_extraction'].apply(clean_technique_name)
extraction_df.head(7)

In [ ]:
print(f"Total unique techniques: {extraction_df['cleaned_technique'].nunique():,}")

In [ ]:
tech_by_paper = extraction_df.groupby(by='doi', as_index=False).agg({'cleaned_technique': set})
tech_by_paper = tech_by_paper.explode('cleaned_technique')
mask_no_feature_extraction = tech_by_paper['cleaned_technique'] != 'nan'
tech_by_paper = tech_by_paper[mask_no_feature_extraction].reset_index(drop=True)
tech_by_paper.head()

In [ ]:
technique_rank = tech_by_paper['cleaned_technique'].value_counts().reset_index()
technique_rank.columns = ['technique', 'count']
technique_rank['percentage'] = np.round(technique_rank['count'] / technique_rank['count'].sum() * 100, 2)
technique_rank.head(10)

In [ ]:
import textwrap

def break_long_names(name, max_len=22, max_lines=3):
    wrapped = textwrap.wrap(
        name,
        width=max_len,
        break_long_words=False,
        break_on_hyphens=False
    )

    # Limit number of lines
    if len(wrapped) > max_lines:
        wrapped = wrapped[:max_lines]
        wrapped[-1] += "…"

    return "\n".join(wrapped)

In [ ]:
top10_techniques = technique_rank.head(10)
from matplotlib.ticker import MaxNLocator


# Seaborn style
sns.set_style("whitegrid")

# Figure size (single-column journal size)
plt.figure(figsize=(9, 6))

# Calculate percentages
percentages = top10_techniques['percentage']

# Bar plot
ax = sns.barplot(
    x=top10_techniques['count'],
    y=top10_techniques['technique'],
    color="#4C72B0",
    edgecolor="black"
)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# Labels and title
ax.set_title("Most Used Feature Extraction Techniques", fontsize=18, pad=12)
ax.set_xlabel("Number of Papers", fontsize=14)
ax.set_ylabel("Feature Extraction Technique", fontsize=14)

# Ticks
ax.tick_params(axis='both', labelsize=11)

# Add value and percentage labels on bars (closer to the end of the bars)
for i, (v, pct) in enumerate(zip(top10_techniques['count'], percentages)):
    ax.text(v - 0.5, i, f"{pct:.1f}%", va='center', ha='right', fontsize=12, fontweight='bold', color='white')

# Improve grid appearance
ax.grid(axis='x', color="#E5E5E5")
ax.grid(axis='y', visible=False)

# Tight layout for clean export
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_PATH, 'feat_ext_barplot.svg'), dpi=600)


In [ ]:
mult_techniques = tech_by_paper.groupby('doi').agg({'cleaned_technique': set})
mult_techniques['num_techniques'] = mult_techniques['cleaned_technique'].apply(lambda x: len(x))
mult_techniques.head()

In [ ]:
num_no_feature = mask_no_feature_extraction.sum()
num_one_technique = mult_techniques[mult_techniques['num_techniques'] == 1].shape[0]
num_multiple_techniques = mult_techniques[mult_techniques['num_techniques'] > 1].shape[0]

In [ ]:
# Create data for the stacked bar plot (single horizontal stacked bar, no y-label)
categories = ['']  # Empty string to hide y-label
counts = [num_no_feature, num_one_technique, num_multiple_techniques]
labels = ['No Feature Extraction', 'One Technique', 'Multiple Techniques']

# Calculate percentages
total_papers = sum(counts)
percentages = [count/total_papers*100 for count in counts]

# Create the horizontal stacked bar plot
plt.figure(figsize=(10, 2.5))

# Colors for each segment
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

lefts = 0
bars = []
for i, (count, color, label, pct) in enumerate(zip(counts, colors, labels, percentages)):
    bar = plt.barh(categories, count, left=lefts, color=color, edgecolor='black', linewidth=1.5, label=f'{label} ({count}, {pct:.1f}%)')
    # Add value and percentage label in the middle of each segment
    plt.text(lefts + count/2, 0, f'{count}\n({pct:.1f}%)', va='center', ha='center', fontsize=12, fontweight='bold', color='white')
    lefts += count

# Customize the plot
plt.title('Distribution of Feature Extraction Usage in Papers', fontsize=16, pad=20)
plt.xlabel('Number of Papers', fontsize=14)
plt.yticks([])  # Remove y-tick label

# Add legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)

# Set x-axis to show integers only
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

# Add grid for better readability
plt.grid(axis='x', alpha=0.3, linestyle='--')

# Adjust layout
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_PATH, 'feat_ext_stacked.svg'), dpi=600)

# Display summary statistics
print(f"Total papers analyzed: {total_papers}")
print(f"Papers with no feature extraction: {num_no_feature} ({percentages[0]:.1f}%)")
print(f"Papers with one technique: {num_one_technique} ({percentages[1]:.1f}%)")
print(f"Papers with multiple techniques: {num_multiple_techniques} ({percentages[2]:.1f}%)")